In [1]:
#IMPORTANDO AS BIBLIOTECAS
import pandas as pd
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import urlopen
import sqlite3
import os
import requests

In [2]:
#Criando as listas para armazenar os dados
ml_url_base = []
ml_urls = []
ml_price = []
ml_seller = []
ml_installment = []
ml_catalog_id = []
ml_catalog_db = []
ml_idetifaction = []
ml_title = []
ml_internacional = []
lista_identification_url = []

In [3]:
#Função para criar os links de busca
def getting_n_creating_mercadolivre_urls(brand):

    Database_path = r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1" + "\Data\\" + brand + "\\" + brand + ".db"

    table = brand + "_products"

    #Criando a Query
    query = "SELECT * FROM " + table

    #Entrando dentro do databse
    connection = sqlite3.connect(Database_path)

    #Criando o dataset em brando
    df = pd.read_sql_query(query, connection)

    #Passando todo o Dataframe para LowerCase
    df = df.apply(lambda x: x.astype(str).str.lower())

    #Arrumando espaços vazios
    # Arrumano os espaços vazios
    df['Product_Name'] = df['Product_Name'].str.replace(" ", "-")

    # Criando uma nova coluna no database com a formatação certa
    df['Urls'] = df['Brand'] + "-" + df['Product_Name']

    # Criando a nova coluna que são as urls de pesquisa
    df['Urls_search'] = "https://lista.mercadolivre.com.br/" + df['Urls'] + "_ITEM*CONDITION_2230284_NoIndex_True?#applied_value_name%3DNovo%"

    return df

In [4]:
Df_Raw = getting_n_creating_mercadolivre_urls('Wacom')
Df_Raw

,Brand,Product_Name,Product_Descrition,Urls,Urls_search
0,wacom,ctl472,ctl 472 - mesa digitalizadora one by wacom peq...,wacom-ctl472,https://lista.mercadolivre.com.br/wacom-ctl472...
1,wacom,ctl4100,ctl4100 - mesa digitalizadora wacom intuos sma...,wacom-ctl4100,https://lista.mercadolivre.com.br/wacom-ctl410...
2,wacom,ctl4100wle0,ctl4100wle0 - mesa digitalizadora wacom intuos...,wacom-ctl4100wle0,https://lista.mercadolivre.com.br/wacom-ctl410...
3,wacom,ctl4100wlk0,ctl4100wlk0 - mesa digitalizadora wacom intuos...,wacom-ctl4100wlk0,https://lista.mercadolivre.com.br/wacom-ctl410...
4,wacom,ctl6100wlk0,ctl6100wlk0 - mesa digitalizadora wacom intuos...,wacom-ctl6100wlk0,https://lista.mercadolivre.com.br/wacom-ctl610...
5,wacom,dtc133w0a,dtc133w0a - mesa digitalizadora wacom one crea...,wacom-dtc133w0a,https://lista.mercadolivre.com.br/wacom-dtc133...
6,wacom,pth460,pth460 - mesa digitalizadora wacom intuos pro ...,wacom-pth460,https://lista.mercadolivre.com.br/wacom-pth460...
7,wacom,dtk1660k0a,dtk1660k0a - display interativo wacom cintiq 1...,wacom-dtk1660k0a,https://lista.mercadolivre.com.br/wacom-dtk166...
8,wacom,pth660,pth660 - mesa digitalizadora wacom intuos pro ...,wacom-pth660,https://lista.mercadolivre.com.br/wacom-pth660...
9,wacom,ctl6100wle0,ctl6100wle0 - mesa digitalizadora wacom intuos...,wacom-ctl6100wle0,https://lista.mercadolivre.com.br/wacom-ctl610...


In [5]:
Df_Raw['Urls_search'][0]

'https://lista.mercadolivre.com.br/wacom-ctl472_ITEM*CONDITION_2230284_NoIndex_True?#applied_value_name%3DNovo%'

In [21]:
urls = ['https://lista.mercadolivre.com.br/Toshiba-HD-externo-Canvio-Basics_ITEM*CONDITION_2230284_NoIndex_True?#applied_value_name%3DNovo%','https://lista.mercadolivre.com.br/HD-Externo-Seagate-Expansion_ITEM*CONDITION_2230284_NoIndex_True?#applied_value_name%3DNovo%']

In [5]:
def search_links(url):
    # Colocando URL commo Global
    global ml_urls

    # Colocando um tempo mínimo de 3 segundos
    time.sleep(3)

    # Fazendo o response
    response = urlopen(url)
    html = response.read()

    # Criando o BeautifulSoup
    BS = BeautifulSoup(html, 'html.parser')

    # Pegando todos os links da página
    for link in BS.find_all("a", href=True):
        ml_urls.append(link['href'])

    # Pegando as próximas páginas
    try:
        next_page_link = BS.find_all(class_='andes-pagination__arrow-title')[-1].text

        # Vendo se na seta está escrito 'Seguinte'
        if next_page_link == 'Seguinte':
            next_url = BS.find_all(class_='andes-pagination__link ui-search-link')[-1]['href']

            # Realizando o loop da função com o link da próxima página
            search_links(next_url)
    except:
        pass

    # Limpando as urls
    ml_urls = [s for s in ml_urls if 'tracking_id' in s]
    ml_urls = [s for s in ml_urls if not 'cabo' in s]
    ml_urls = [s for s in ml_urls if not 'luva' in s]
    ml_urls = [s for s in ml_urls if not 'pontas' in s]
    ml_urls = [s for s in ml_urls if not 'suporte' in s]
    ml_urls = [s for s in ml_urls if not 'caneta' in s]
    ml_urls = [s for s in ml_urls if not 'folha' in s]
    ml_urls = [s for s in ml_urls if not 'controle' in s]
    #ml_urls = [s for s in ml_urls if 'produto' in s]
    #ml_urls = [s for s in ml_urls if 'wacom' in s]

    # Tirando as duplicadas
    ml_urls = list(dict.fromkeys(ml_urls))

In [7]:
for url in tqdm(Df_Raw['Urls_search']):
    search_links(url)

100%|██████████| 10/10 [01:25<00:00,  8.55s/it]


In [8]:
search_links(url)

In [46]:
response = urlopen(ml_urls[0])
html = response.read()

#Criando o soup
BS = BeautifulSoup(html, 'html.parser')

In [47]:
BS.find(class_='andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact').text

'283 realesR$283'

In [50]:
div = BS.find('div',attrs={'class':'ui-pdp-price__subtitles'})
div.find('span')
for span in div.find_all("span"):
    print(span)

<span class="ui-pdp-color--BLACK ui-pdp-family--REGULAR">em</span>
<span class="andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact" style="font-size:18px"><span class="andes-visually-hidden">31 reales con 44 centavos</span><span aria-hidden="true" class="andes-money-amount__currency-symbol">R$</span><span aria-hidden="true" class="andes-money-amount__fraction">31</span><span aria-hidden="true" class="andes-money-amount__cents andes-money-amount__cents--superscript-18" style="font-size:10px;margin-top:2px">44</span></span>
<span class="andes-visually-hidden">31 reales con 44 centavos</span>
<span aria-hidden="true" class="andes-money-amount__currency-symbol">R$</span>
<span aria-hidden="true" class="andes-money-amount__fraction">31</span>
<span aria-hidden="true" class="andes-money-amount__cents andes-money-amount__cents--superscript-18" style="font-size:10px;margin-top:2px">44</span>


In [52]:
div.text

'em 9x 31 reales con 44 centavosR$3144 sem juros'

In [31]:
try:
    div = BS.find('div',attrs={'class':'ui-pdp-price__subtitles'})
    installment = div.text
    ml_installment.append(installment)
except:
    installment = BS.find(class_='ui-pdp-color--GREEN ui-pdp-size--MEDIUM ui-pdp-family--REGULAR').text
    ml_installment.append(installment)


<!DOCTYPE html>

<html lang="pt-BR">
<head><link href="https://www.google-analytics.com" rel="preconnect"/><link href="https://www.google.com" rel="preconnect"/><link href="https://data.mercadolibre.com" rel="preconnect"/><link href="https://http2.mlstatic.com" rel="preconnect"/><link href="https://stats.g.doubleclick.net" rel="preconnect"/><link href="https://analytics.mercadolivre.com.br" rel="preconnect"/><link href="https://analytics.mercadolivre.com" rel="preconnect"/><link href="https://www.google.com.br" rel="preconnect"/><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-689ffbd95eae88e39ac","applicationID":"1588198355","applicationTime":262.914878,"transactionName":"YlZQYEVZC0QEV0BZV1scd0xHSgBEFl5HH39wZx0bDUgXWAFBV0RxURsab1YVH3ZIbmlLC0geaQQaATgcTBweGRE=","queueTime":0,"ttGuid":"8b66ff82ce8b32dd","agentToken":null}; (window.NREUM||(NREUM={})).init={privacy:{cookies_enabl

In [8]:
def search_attributes(url):
    #Criando tempo
    time.sleep(2)

    #Fazendo o response jh
    response = urlopen(url)
    html = response.read()

    #Criando o soup
    BS = BeautifulSoup(html, 'html.parser')

    #Buscando o preço
    try:
        price = BS.find(class_='andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact').text
        ml_price.append(price)
    except:
        ml_price.append("ERRO")

    #Buscando o installment
    try:
        div = BS.find('div',attrs={'class':'ui-pdp-price__subtitles'})
        installment = div.text
        ml_installment.append(installment)
    except:
        try:
            installment = BS.find(class_='ui-pdp-color--GREEN ui-pdp-size--MEDIUM ui-pdp-family--REGULAR').text
            ml_installment.append(installment)
        except:
            ml_installment.append("ERRO")

    #Catalog
    try:
        catalog = BS.find(class_='ui-pdp-other-sellers__link')['href']
        ml_catalog_id.append(catalog)
    except:
        ml_catalog_id.append("NORMAL")

    #Título
    try:
        title = BS.find(class_='ui-pdp-title')
        ml_title.append(title.text)
    except:
        ml_title.append("Erro")

    #Vendedor
    try:
        seller_link = BS.find(class_='ui-pdp-media__action ui-box-component__action')['href']
    except:
        seller_link = "Erro"

    #Internacional
    try:
        internacional_text = BS.find(class_='ui-pdp-icon ui-pdp-icon--cbt-summary ui-pdp-color--BLUE_700').text
        ml_internacional.append(internacional_text)
    except:
        ml_internacional.append("ERRO")


    try:
        #Entrando na página do vendedor
        response = urlopen(seller_link)
        html = response.read()

        #Criando o soup
        BS = BeautifulSoup(html, 'html.parser')

        #Achando o nome do seller
        seller_name = BS.find(class_='store-info__name').text

        #Append do nome do seller
        ml_seller.append(seller_name)
    except:
        ml_seller.append(seller_link)

In [9]:
for url in tqdm(ml_urls):
    search_attributes(url)

100%|██████████| 397/397 [27:06<00:00,  4.10s/it]


In [10]:
 def create_dataframe(url,seller,price,installment,catalogo,internacional):
    Dataset = pd.DataFrame()

    Dataset['URL'] = url
    Dataset['DATE'] = pd.to_datetime('today', errors='ignore').date()
    Dataset['PRODUCT'] = ml_title

    Dataset['MARKETPLACE'] = 'MERCADO LIVRE'
    Dataset['SELLER'] = seller

    Dataset['Price'] = price
    Dataset['PRICE_1'] = Dataset['Price'].str.partition("reales")[0]
    Dataset["PRICE_2"] = Dataset["Price"].str.partition(" con")[2].str.partition("centavos")[0].str.partition(" ")[2].str.partition(" centavo")[0]

    Dataset['PRICE'] = Dataset['PRICE_1'] + "," + Dataset['PRICE_2']
    Dataset['PRICE'] = Dataset['PRICE'].str.replace(" ","")

    Dataset['CATALOGO'] = catalogo
    Dataset['INTERNACIONAL'] = internacional

    Dataset['Installment'] = installment

    # Arrumando a coluna de installment
    Dataset['PARCEL'] = Dataset['Installment'].str.partition("x")[0]
    #Dataset['PARCEL'] = Dataset['PARCEL'].str.extract("(\d+)").astype(int)
    Dataset["reais"] = Dataset["Installment"].str.partition("reales")[0].str.partition("x")[2]
    Dataset["moedas"] = Dataset["Installment"].str.partition(" con")[2].str.partition("centavos")[0].str.partition(" ")[2].str.partition(" centavo")[0]
    # Dataset['Installment'] = Dataset['reais'] + "." + Dataset['moedas']

    Dataset['INSTALLMENT'] = Dataset['reais'] + "," + Dataset['moedas']
    Dataset['INSTALLMENT'] = Dataset['INSTALLMENT'].str.replace(" ","")

    #Dataset['Catalog'] = catalogo

    Dataset['ASIN'] = Dataset['URL'].str.partition("/")[2].str.partition("/")[2].str.partition("/")[2].str.partition("-")[0]
    Dataset['ASIN_2'] = Dataset['URL'].str.partition("/")[2].str.partition("/")[2].str.partition("/")[2].str.partition("-")[2].str.partition("-")[0]

    Dataset['ASIN_CORRETO'] = Dataset['ASIN'] + Dataset['ASIN_2']

    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/HUAWEIOFICIAL?brandId=3562",'HUAWEI LOJA OFICIAL')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MPCEL+MOBILE?brandId=3430",'Mpcel Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MPCEL+MOBILE?brandId=3562",'MPCEL MOBILE')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/ONOFREELETROSERRA?brandId=2156",'Onofre Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/ZOOM_STORE?brandId=3598",'Zoom Store Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/FASTSHOP+OFICIAL?brandId=942",'Fast Shop Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MERCADOLIVRE+ELETRONICOS?brandId=2707",'Mercado Livre Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MERCADOLIVRE+ELETRONICOS?brandId=2707", 'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST?brandId=866", 'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST-STORE+PREMIUM?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST-STORE+SP?brandId=866", 'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/SUNTEKSTOREBR?brandId=2203",'Suntek Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/SUNTEKSTOREBR?brandId=2203",'Suntek Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/OLIST?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/OLIST-STORE+PREMIUM?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/OLIST-STORE+SP?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/E-SPOT?brandId=4166",'Wacom Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/IBYTE.?brandId=3979",'Ibyte Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/INPOWER+INFO?brandId=1655",'INPOWER Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/PICHAUINFORMATICA?brandId=1436",'Pichau Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/PRIMETEK+COMPUTADORES?brandId=2255",'Primetek Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/VLSTORE+INFORMATICA?brandId=3629", 'Leva Digital Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/MIMI2231343?brandId=3396", 'Miranda Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/MIMI2231343?brandId=3396",'Miranda Loja Oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/TUDOABE%C3%87A?brandId=1664",'Tudo à Beça Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST-STORE+BA?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/OLIST-STORE?brandId=866",'Olist Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://www.mercadolivre.com.br/perfil/E-SPOT?brandId=3574",'E-spot Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/BRING.IT2?brandId=1818",'Bring It Loja oficial')
    Dataset['SELLER'] = Dataset['SELLER'].replace("https://perfil.mercadolivre.com.br/A+P%C3%81GINA?brandId=1256",'A Página Loja oficial')


    Dataset.drop(['ASIN','ASIN_2'], axis=1, inplace=True)

    Dataset = Dataset[['DATE','URL','MARKETPLACE','SELLER','PRICE','PARCEL','INSTALLMENT','ASIN_CORRETO','PRODUCT','CATALOGO','INTERNACIONAL']]

    return Dataset

In [11]:
dataset_mercadolivre = create_dataframe(ml_urls,ml_seller,ml_price,ml_installment,ml_catalog_id,ml_internacional)

In [12]:
dataset_mercadolivre

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ASIN_CORRETO,PRODUCT,CATALOGO,INTERNACIONAL
0,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,SHOP EV,"275,",em 9,"30,56",mesadigitalizadora,Mesa digitalizadora Wacom One by Wacom CTL-472...,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
1,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,TECHNOLINKS SOLUTIONS,"523,90",em 10,"52,39",mesadigitalizadora,Mesa digitalizadora Wacom Intuos Small black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
2,2022-08-01,https://produto.mercadolivre.com.br/MLB-165076...,MERCADO LIVRE,INPOWER Loja Oficial,"299,",ou em 12,"28,99",MLB1650763846,Mesa Digitalizadora Wacom One By Wacom Ctl472l P,NORMAL,ERRO
3,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,YANAZE OFICIAL,"448,80",em 12,"43,51",mesadigitalizadora,Mesa digitalizadora 10moons 1060Plus black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
4,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,VB.TECH,"204,",ou em 12,"19,78",mesadigitalizadora,Mesa digitalizadora Huion 420 black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
...,...,...,...,...,...,...,...,...,...,...,...
392,2022-08-01,https://produto.mercadolivre.com.br/MLB-269653...,MERCADO LIVRE,Erro,"1561,14",em 12,"151,35",MLB2696530325,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...,NORMAL,ERRO
393,2022-08-01,https://produto.mercadolivre.com.br/MLB-164592...,MERCADO LIVRE,INPOWER Loja Oficial,"1699,",em 10,"169,90",MLB1645929088,Mesa Digitalizadora Wacom Intuos M Bluetooth P...,NORMAL,ERRO
394,2022-08-01,https://produto.mercadolivre.com.br/MLB-195069...,MERCADO LIVRE,W-TECH-IMPORTS,"1797,",em 12,"174,22",MLB1950693367,Mesa Digitalizadora Wacom Intuos Ctl6100wle0 B...,NORMAL,ERRO
395,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,GALUDOSTORE,"700,",em 12,"67,86",mesadigitalizadora,Mesa digitalizadora Wacom Intuos S com Bluet...,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO


In [13]:
#Retirando as duplicadas
dataset_mercadolivre = dataset_mercadolivre.drop_duplicates(subset=['SELLER','PRICE','PARCEL','INSTALLMENT','PRODUCT'])
dataset_mercadolivre

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ASIN_CORRETO,PRODUCT,CATALOGO,INTERNACIONAL
0,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,SHOP EV,"275,",em 9,"30,56",mesadigitalizadora,Mesa digitalizadora Wacom One by Wacom CTL-472...,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
1,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,TECHNOLINKS SOLUTIONS,"523,90",em 10,"52,39",mesadigitalizadora,Mesa digitalizadora Wacom Intuos Small black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
2,2022-08-01,https://produto.mercadolivre.com.br/MLB-165076...,MERCADO LIVRE,INPOWER Loja Oficial,"299,",ou em 12,"28,99",MLB1650763846,Mesa Digitalizadora Wacom One By Wacom Ctl472l P,NORMAL,ERRO
3,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,YANAZE OFICIAL,"448,80",em 12,"43,51",mesadigitalizadora,Mesa digitalizadora 10moons 1060Plus black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
4,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,VB.TECH,"204,",ou em 12,"19,78",mesadigitalizadora,Mesa digitalizadora Huion 420 black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
...,...,...,...,...,...,...,...,...,...,...,...
388,2022-08-01,https://produto.mercadolivre.com.br/MLB-195069...,MERCADO LIVRE,W-TECH-IMPORTS,"1897,",em 10,"189,70",MLB1950697616,Mesa Digitalizadora Wacom Intuos Ctl6100wle0 B...,NORMAL,ERRO
390,2022-08-01,https://produto.mercadolivre.com.br/MLB-271647...,MERCADO LIVRE,Erro,"1659,90",em 10,"165,99",MLB2716479154,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...,NORMAL,ERRO
392,2022-08-01,https://produto.mercadolivre.com.br/MLB-269653...,MERCADO LIVRE,Erro,"1561,14",em 12,"151,35",MLB2696530325,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...,NORMAL,ERRO
394,2022-08-01,https://produto.mercadolivre.com.br/MLB-195069...,MERCADO LIVRE,W-TECH-IMPORTS,"1797,",em 12,"174,22",MLB1950693367,Mesa Digitalizadora Wacom Intuos Ctl6100wle0 B...,NORMAL,ERRO


In [14]:
Df_catalog = dataset_mercadolivre[dataset_mercadolivre['CATALOGO'] != 'NORMAL']
Df_catalog

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ASIN_CORRETO,PRODUCT,CATALOGO,INTERNACIONAL
0,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,SHOP EV,"275,",em 9,"30,56",mesadigitalizadora,Mesa digitalizadora Wacom One by Wacom CTL-472...,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
1,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,TECHNOLINKS SOLUTIONS,"523,90",em 10,"52,39",mesadigitalizadora,Mesa digitalizadora Wacom Intuos Small black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
3,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,YANAZE OFICIAL,"448,80",em 12,"43,51",mesadigitalizadora,Mesa digitalizadora 10moons 1060Plus black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
4,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,VB.TECH,"204,",ou em 12,"19,78",mesadigitalizadora,Mesa digitalizadora Huion 420 black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
6,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,ROSANAROSENTINO,"217,",ou em 12,"21,04",mesadigitalizadora,Mesa digitalizadora Huion Inspiroy H430P black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
8,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,KEJING,"248,",ou em 12,"24,04",mesadigitalizadora,Mesa digitalizadora Huion HS64 black,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
69,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,SHOPAR.COM.BR,"744,90",em 12,"72,22",mesadigitalizadora,Mesa digitalizadora Wacom Intuos S com Bluet...,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
89,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,GALUDOSTORE,"700,",em 12,"67,86",mesadigitalizadora,Mesa digitalizadora Wacom Intuos S com Bluet...,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
100,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,SHOPAR.COM.BR,"3464,90",em 10,"346,49",mesadigitalizadora,Mesa digitalizadora Wacom One black e white,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO
155,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,Erro,"1959,90",em 10,"195,99",mesadigitalizadora,Mesa digitalizadora Wacom Intuos Pro Small PT...,https://www.mercadolivre.com.br/mesa-digitaliz...,ERRO


In [15]:
seller_catalog = []
price_catalog = []
installment_catalog = []
link_catalog = []
product_catalog = []

In [45]:
def search_catalog(url):
    time.sleep(3)
    #Entrando no html
    response = requests.get(url)

    #Pegando o html
    html = response.text

    #Criando o Soup
    Soup = BeautifulSoup(html, 'html.parser')

    produto = Soup.find(class_='ui-pdp-title').text

    #Pegando o preço
    for price in Soup.find_all(class_='andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact'):
        price_catalog.append(price.text)

    #Pegando as parcelas
    try:
        for price in Soup.find_all(class_='ui-pdp-family--REGULAR ui-pdp-media__title'):
            installment_catalog.append(price.text)
    except:
        for price in Soup.find_all(class_='ui-pdp-family--REGULAR ui-pdp-media__title'):
            installment_catalog.append(price.text)

    #Pegando os vendedores
    for seller in Soup.find_all(class_='ui-pdp-color--BLUE ui-pdp-family--REGULAR'):
        seller_catalog.append(seller.text)
        link_catalog.append(url)
        product_catalog.append(produto)

    try:
        #Pegando a próxima página


        #Fazendo o try da função com a nova url
        search_catalog(Soup.find(class_='andes-pagination__link', attrs={"title":'Seguinte'})['href'])
        print("Nova página")
    except:
        pass

In [46]:
urls = ["https://www.mercadolivre.com.br/disco-rigido-externo-toshiba-canvio-basics-hdtb410xk3aa-1tb-preto/p/MLB10687522/s?pdp_filters=ITEM_CONDITION:2230284"]

In [50]:
for url in tqdm(urls):
    search_catalog(url)

100%|██████████| 1/1 [00:14<00:00, 14.27s/it]

Nova página
Nova página
Nova página


In [ ]:
search_catalog(urls)

In [51]:
print(len(seller_catalog))
print(len(price_catalog))
print(len(installment_catalog))
print(len(link_catalog))
print(len(product_catalog))


28
28
28
28
28


In [43]:
seller_catalog.append('erro')
link_catalog.append('erro')
product_catalog.append('erro')

In [48]:
#Catalog
seller_catalog = []
price_catalog = []
installment_catalog = []
link_catalog = []
product_catalog = []

In [52]:
Catalog_dataframe = pd.DataFrame()

Catalog_dataframe['URL'] = link_catalog
Catalog_dataframe['SELLER'] = seller_catalog

Catalog_dataframe['PRICE_BRUTO'] = price_catalog
Catalog_dataframe['PRICE_REAL'] = Catalog_dataframe['PRICE_BRUTO'].str.partition(" reales")[0]
Catalog_dataframe['PRICE_CENTAVOS'] = Catalog_dataframe['PRICE_BRUTO'].str.partition('con ')[2].str.partition(" centavos")[0]

Catalog_dataframe['PRICE'] = Catalog_dataframe['PRICE_REAL'] + "," + Catalog_dataframe['PRICE_CENTAVOS']

Catalog_dataframe['INSTALLMENT_BRUTO'] = installment_catalog
Catalog_dataframe['PARCEL'] = Catalog_dataframe['INSTALLMENT_BRUTO'].str.partition('x')[0]

Catalog_dataframe['INSTALLMENT_REAL'] = Catalog_dataframe['INSTALLMENT_BRUTO'].str.partition('x ')[2].str.partition(" reales")[0]
Catalog_dataframe['INSTALLMENT_CENTAVO'] = Catalog_dataframe['INSTALLMENT_BRUTO'].str.partition('con ')[2].str.partition(' centavos')[0]

Catalog_dataframe['INSTALLMENT'] = Catalog_dataframe['INSTALLMENT_REAL'] + "," + Catalog_dataframe['INSTALLMENT_CENTAVO']

Catalog_dataframe['DATE'] = pd.to_datetime('today', errors='ignore').date()

Catalog_dataframe['MARKETPLACE'] = 'MERCADO LIVRE'

Catalog_dataframe = Catalog_dataframe.drop(columns=['PRICE_BRUTO','PRICE_REAL','PRICE_CENTAVOS','INSTALLMENT_BRUTO','INSTALLMENT_REAL','INSTALLMENT_CENTAVO'])

Catalog_dataframe['ASIN_CORRETO'] = Catalog_dataframe['URL'].str.partition('/p/')[2].str.partition('/')[0]

Catalog_dataframe['PRODUCT'] = product_catalog

Catalog_dataframe['CATALOGO'] = 'CATALOGO'

Catalog_dataframe = Catalog_dataframe[['DATE','URL','MARKETPLACE','SELLER','PRICE','PARCEL','INSTALLMENT','ASIN_CORRETO','PRODUCT','CATALOGO']]

Catalog_dataframe

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ASIN_CORRETO,PRODUCT,CATALOGO
0,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,CASA HELENA,"319,",12,"30,93",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
1,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,TNTINFOLOJA.COM.BR,"329,",10,"32,90",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
2,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,123 Comprou,"339,90",12,"32,95",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
3,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,CTBASHOP.,"344,65",12,"33,41",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
4,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,CTBASHOPCURITIBA,"344,65",12,"33,41",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
5,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,CTBASHOP.,"359,90",10,"35,99",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
6,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,CTBASHOPCURITIBA,"359,90",10,"35,99",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
7,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,MEGA GAMESS,"365,",10,"36,50",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
8,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,LOJAVISIVO,"369,",12,"35,77",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO
9,2022-08-01,https://www.mercadolivre.com.br/disco-rigido-e...,MERCADO LIVRE,A6 COMERCIAL,"377,58",10,"37,76",MLB10687522,Disco rígido externo Toshiba Canvio Basics HDT...,CATALOGO


In [37]:
dataset_mercadolivre = dataset_mercadolivre[dataset_mercadolivre['CATALOGO'] == 'NORMAL']
dataset_mercadolivre

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ASIN_CORRETO,PRODUCT,CATALOGO,INTERNACIONAL
2,2022-08-01,https://produto.mercadolivre.com.br/MLB-165076...,MERCADO LIVRE,INPOWER Loja Oficial,"299,",ou em 12,"28,99",MLB1650763846,Mesa Digitalizadora Wacom One By Wacom Ctl472l P,NORMAL,ERRO
5,2022-08-01,https://produto.mercadolivre.com.br/MLB-188374...,MERCADO LIVRE,SHOPAR.COM.BR,"299,90",ou em 12,"29,08",MLB1883744978,Mesa Digitalizadora Wacom One By Wacom Ctl472 ...,NORMAL,ERRO
7,2022-08-01,https://produto.mercadolivre.com.br/MLB-121742...,MERCADO LIVRE,INPOWER Loja Oficial,"322,",em 10,"32,20",MLB1217422686,Mesa Digitalizadora Wacom One By Wacom Ctl472l P,NORMAL,ERRO
9,2022-08-01,https://produto.mercadolivre.com.br/MLB-268509...,MERCADO LIVRE,SHOP EV,"275,",em 9,"30,56",MLB2685092467,Mesa Digitalizadora Wacom One By Wacom Ctl-472,NORMAL,ERRO
10,2022-08-01,https://produto.mercadolivre.com.br/MLB-188370...,MERCADO LIVRE,SHOPAR.COM.BR,"314,90",em 10,"31,49",MLB1883704185,Mesa Digitalizadora Wacom One By Wacom Ctl472 ...,NORMAL,ERRO
...,...,...,...,...,...,...,...,...,...,...,...
388,2022-08-01,https://produto.mercadolivre.com.br/MLB-195069...,MERCADO LIVRE,W-TECH-IMPORTS,"1897,",em 10,"189,70",MLB1950697616,Mesa Digitalizadora Wacom Intuos Ctl6100wle0 B...,NORMAL,ERRO
390,2022-08-01,https://produto.mercadolivre.com.br/MLB-271647...,MERCADO LIVRE,Erro,"1659,90",em 10,"165,99",MLB2716479154,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...,NORMAL,ERRO
392,2022-08-01,https://produto.mercadolivre.com.br/MLB-269653...,MERCADO LIVRE,Erro,"1561,14",em 12,"151,35",MLB2696530325,Mesa Digitalizadora Wacom Intuos Bluetooth Méd...,NORMAL,ERRO
394,2022-08-01,https://produto.mercadolivre.com.br/MLB-195069...,MERCADO LIVRE,W-TECH-IMPORTS,"1797,",em 12,"174,22",MLB1950693367,Mesa Digitalizadora Wacom Intuos Ctl6100wle0 B...,NORMAL,ERRO


In [21]:
Df_final = pd.concat([dataset_mercadolivre,Catalog_dataframe])
Df_final

,DATE,URL,MARKETPLACE,SELLER,PRICE,PARCEL,INSTALLMENT,ASIN_CORRETO,PRODUCT,CATALOGO,INTERNACIONAL
2,2022-08-01,https://produto.mercadolivre.com.br/MLB-165076...,MERCADO LIVRE,INPOWER Loja Oficial,"299,",ou em 12,"28,99",MLB1650763846,Mesa Digitalizadora Wacom One By Wacom Ctl472l P,NORMAL,ERRO
5,2022-08-01,https://produto.mercadolivre.com.br/MLB-188374...,MERCADO LIVRE,SHOPAR.COM.BR,"299,90",ou em 12,"29,08",MLB1883744978,Mesa Digitalizadora Wacom One By Wacom Ctl472 ...,NORMAL,ERRO
7,2022-08-01,https://produto.mercadolivre.com.br/MLB-121742...,MERCADO LIVRE,INPOWER Loja Oficial,"322,",em 10,"32,20",MLB1217422686,Mesa Digitalizadora Wacom One By Wacom Ctl472l P,NORMAL,ERRO
9,2022-08-01,https://produto.mercadolivre.com.br/MLB-268509...,MERCADO LIVRE,SHOP EV,"275,",em 9,"30,56",MLB2685092467,Mesa Digitalizadora Wacom One By Wacom Ctl-472,NORMAL,ERRO
10,2022-08-01,https://produto.mercadolivre.com.br/MLB-188370...,MERCADO LIVRE,SHOPAR.COM.BR,"314,90",em 10,"31,49",MLB1883704185,Mesa Digitalizadora Wacom One By Wacom Ctl472 ...,NORMAL,ERRO
...,...,...,...,...,...,...,...,...,...,...,...
290,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,Wacom,"4652,90",10,"465,29",MLB9050721,Mesa digitalizadora Wacom Intuos Pro Large PT...,CATALOGO,NaN
291,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,STORE_JB,"4699,90",10,"469,99",MLB9050721,Mesa digitalizadora Wacom Intuos Pro Large PT...,CATALOGO,NaN
292,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,DJI OFFICIAL,"4799,90",10,"479,99",MLB9050721,Mesa digitalizadora Wacom Intuos Pro Large PT...,CATALOGO,NaN
293,2022-08-01,https://www.mercadolivre.com.br/mesa-digitaliz...,MERCADO LIVRE,W-TECH-IMPORTS,"5297,",12,"513,54",MLB9050721,Mesa digitalizadora Wacom Intuos Pro Large PT...,CATALOGO,NaN


In [25]:
Df_final.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Software_Artemis\Data\Brands_Downloads\Wacom\Mercado_Wacom.xlsx", index=False)

In [53]:
Catalog_dataframe.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Software_Artemis\Data\Brand_Search_Test\Mercado_catalogo_Livre_Toshiba.xlsx", index=False)

In [120]:
dataset_mercadolivre.loc[dataset_mercadolivre['URL'] == 'https://www.mercadolivre.com.br/mesa-digitalizadora-wacom-one-by-wacom-ctl-472-preta-e-vermelha/p/MLB18062496?pdp_filters=ITEM_CONDITION:2230284#searchVariation=MLB18062496&position=1&search_layout=stack&type=product&tracking_id=f5dfdfdd-ff19-47a0-b9ad-c91a218efa4d','PRODUCT']

0    Mesa digitalizadora Wacom One by Wacom CTL-472...
Name: PRODUCT, dtype: object

In [118]:
dataset_mercadolivre['URL'][0]

'https://www.mercadolivre.com.br/mesa-digitalizadora-wacom-one-by-wacom-ctl-472-preta-e-vermelha/p/MLB18062496?pdp_filters=ITEM_CONDITION:2230284#searchVariation=MLB18062496&position=1&search_layout=stack&type=product&tracking_id=f5dfdfdd-ff19-47a0-b9ad-c91a218efa4d'

In [91]:
Catalog_dataframe.dtypes

URL                  object
SELLER               object
PRICE_BRUTO          object
INSTALLMENT_BRUTO    object
PRICE_REAL           object
PRICE_CENTAVOS       object
PRICE                object
dtype: object

In [15]:
import requests

In [128]:
response = requests.get('https://www.mercadolivre.com.br/mesa-digitalizadora-wacom-one-by-wacom-ctl-472-preta-e-vermelha/p/MLB18062496/s?pdp_filters=ITEM_CONDITION%3A2230284&page=2')
html = response.text

Soup = BeautifulSoup(html, 'html.parser')

In [20]:
for price in Soup.find_all(class_='andes-money-amount ui-pdp-price__part andes-money-amount--cents-superscript andes-money-amount--compact'):
    print(price.text)

219 reales con 98 centavosR$21998
221 reales con 35 centavosR$22135
228 reales con 89 centavosR$22889
229 reales con 89 centavosR$22989
229 reales con 90 centavosR$22990
230 realesR$230
232 realesR$232
237 realesR$237
237 reales con 62 centavosR$23762
239 reales con 77 centavosR$23977


In [21]:
for price in Soup.find_all(class_='ui-pdp-media ui-pdp-payment ui-pdp-payment--md ui-pdp-s-payment ui-pdp-color--BLACK'):
    print(price.text)

12x 21 reales con 33 centavosR$2133
12x 21 reales con 46 centavosR$2146
12x 22 reales con 19 centavosR$2219
12x 22 reales con 29 centavosR$2229
12x 22 reales con 29 centavosR$2229
12x 22 reales con 49 centavosR$2249
12x 22 reales con 98 centavosR$2298
12x 23 reales con 04 centavosR$2304


In [129]:
for price in Soup.find_all(class_='ui-pdp-color--BLUE ui-pdp-family--REGULAR'):
    print(price.text)

SHOPAR.COM.BR
X-MANIA.
INPOWER
NLISBOA ELETRONICOS
NLISBOA ELETRONICOS
O REI.STORE
CATHITEC
LM.COM
-VSP-
Olist


In [139]:
response = requests.get(Soup.find(class_='andes-pagination__link', attrs={"title":'Seguinte'})['href'])
response.status_code

200

In [44]:
value = Soup.find(class_='andes-pagination__link')['href']

'h'

In [15]:
dataset_mercadolivre.to_excel(r"C:\Users\pedro\Documents\Turte Brand Protection\Turtle_Thinker_Alpha_0.1\Data\Wacom\Files\MercadoL_Wacom.xlsx", index=False)

In [7]:
#Fazendo o response jh
response = urlopen("https://produto.mercadolivre.com.br/MLB-1680322012-caneta-digital-wacom-lp190-para-wacomctl-472-ctl-672-ctl-690-_JM?searchVariation=65783671110#searchVariation=65783671110&position=1&search_layout=grid&type=item&tracking_id=10a18886-f499-40f6-98b8-01d7d80aaa0d")
html = response.read()

#Criando o soup
BS = BeautifulSoup(html, 'html.parser')

In [8]:
BS.find(class_='ui-pdp-icon ui-pdp-icon--cbt-summary ui-pdp-color--BLUE_700').text

'Compra internacional'

In [6]:
BS.find(class_='ui-pdp-other-sellers__link')['href']

'https://www.mercadolivre.com.br/mesa-digitalizadora-wacom-one-by-wacom-ctl-472-preta-e-vermelha/p/MLB18062496/s?pdp_filters=category:MLB6711'